In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import time
%matplotlib inline

### Join dataset

#### Download

Download all four datasets from here: https://crtm.maps.arcgis.com/apps/MinimalGallery/index.html?appid=a60bb2f0142b440eadee1a69a11693fc
and store in *data\raw\*

**Data**

In [2]:
ind = pd.read_excel (r'..\data\raw\EDM2018INDIVIDUOS.xlsx')
hog = pd.read_excel (r'..\data\raw\EDM2018HOGARES.xlsx')
via = pd.read_excel (r'..\data\raw\EDM2018VIAJES.xlsx', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})
etap = pd.read_excel (r'..\data\raw\EDM2018XETAPAS.xlsx')

via.set_index(["ID_HOGAR", "ID_IND", "ID_VIAJE"], inplace = True)
ind.set_index(["ID_HOGAR", "ID_IND"], inplace = True)
hog.set_index("ID_HOGAR", inplace = True)
etap.set_index(["ID_HOGAR", "ID_IND", "ID_VIAJE"], inplace =True)

In [3]:
people = hog.join(ind, lsuffix = "_hog", rsuffix = "_ind")
trips = people.join(via, rsuffix = "_via")
legs = trips.join(etap, rsuffix = "_etap")

people.to_csv(r'..\data\interim\people.csv',  index = False)
trips.to_csv(r'..\data\interim\trips.csv',  index = False)
legs.to_csv(r'..\data\interim\legs.csv',  index = False)

**Codes**

In [ ]:
codes_hog = pd.read_excel (r'..\data\raw\EDM2018HOGARES.xlsx', sheet_name = 1)
codes_ind = pd.read_excel (r'..\data\raw\EDM2018INDIVIDUOS.xlsx', sheet_name = 1)
codes_via = pd.read_excel (r'..\data\raw\EDM2018VIAJES.xlsx', sheet_name = 1)
codes_eta = pd.read_excel (r'..\data\raw\EDM2018XETAPAS.xlsx', sheet_name = 1)

codes = codes_hog.append(codes_ind, ignore_index = True, sort = False
                        ).append(codes_via, ignore_index = True, sort = False
                                ).append(codes_eta, ignore_index = True, sort = False
                                        ).drop(columns = ["Unnamed: 0", "Unnamed: 1"])

codes["VARIABLE"] = codes.VARIABLE.fillna(method = "ffill")
codes["ESPECIFICACIÓN"] = codes["ESPECIFICACIÓN"].fillna(method = "ffill")

codes['CODE'], codes['VALUE'] = codes['VALORES'].str.split(".", 1).str
codes['CODE'], unused = codes.CODE.str.split(" '", 1).str

codes.drop(index = codes[codes.VARIABLE.isna()].index, inplace = True)

codes.to_csv(r'..\data\interim\codes.csv', index = False)

translated codes not programmatically, but manually via google Sheets.

- Translation is stored in data\processed\codes_translated.csv

### create new variables

In [ ]:
# trips = pd.read_csv(r'..\data\interim\trips.csv', dtype = {'VORIHORAINI':str, 'VDESHORAFIN':str})

In [5]:
codes = pd.read_csv(r'..\data\processed\codes_translated.csv', dtype = {'CODE': float})

In [6]:
# set value of sex
code_sex = codes[codes.VARIABLE == "C2SEXO"][['CODE', 'VALUE_en']].rename({'VALUE_en': 'sex'}, axis = 1)
trips = trips.join(code_sex.set_index('CODE'), on = "C2SEXO")

In [7]:
# duration
mins = pd.to_numeric(trips.VDESHORAFIN.str.slice(2)) - pd.to_numeric(trips.VORIHORAINI.str.slice(2))
hours = (pd.to_numeric(trips.VDESHORAFIN.str.slice(0, 2)) - pd.to_numeric(trips.VORIHORAINI.str.slice(0, 2))) * 60
trips["duration"] = mins + hours

In [8]:
# start- and endtime, duration and speed.dt.time
trips['start_time'] = pd.to_datetime(trips.VORIHORAINI, format = '%H%M').dt.time

# TODO: programmatic solution for all times > 24h
trips.loc[trips.VDESHORAFIN == '2500', 'VDESHORAFIN'] = '0100'
trips.loc[trips.VDESHORAFIN == '2505', 'VDESHORAFIN'] = '0105'
trips.loc[trips.VDESHORAFIN == '2515', 'VDESHORAFIN'] = '0115'
trips.loc[trips.VDESHORAFIN == '2530', 'VDESHORAFIN'] = '0130'
trips.loc[trips.VDESHORAFIN == '2600', 'VDESHORAFIN'] = '0200'
trips.loc[trips.VDESHORAFIN == '2800', 'VDESHORAFIN'] = '0400'

trips['end_time'] = pd.to_datetime(trips.VDESHORAFIN, format = '%H%M', errors = 'coerce').dt.time

In [9]:
# speed
trips["speed"] = np.where(trips.duration != 0, # skip if start and Endtime is the same
                       trips.DISTANCIA_VIAJE / (trips.duration / 60), np.NaN)

In [10]:
# trip mode
code_mode = codes[codes['VARIABLE'] == 'MODO_PRIORITARIO'][['CODE', 'VALUE_en']].drop_duplicates()
trips = trips.join(code_mode.set_index('CODE'),  on = 'MODO_PRIORITARIO', how = "left").reset_index()
trips.rename({"VALUE_en": "mode"}, axis = "columns", inplace = True)

# simplify mode
trips["mode_simple"] = trips.MODO_PRIORITARIO
trips.loc[trips.MODO_PRIORITARIO < 10, "mode_simple"] = "public transport"
trips.loc[(trips.MODO_PRIORITARIO > 9) & (trips.MODO_PRIORITARIO < 17), "mode_simple"] = "car"
trips.loc[(trips.MODO_PRIORITARIO > 16) & (trips.MODO_PRIORITARIO < 24), "mode_simple"] = "other"
trips.loc[trips.MODO_PRIORITARIO == 24, "mode_simple"] = "walk"

In [11]:
# trip motive
code_motive = codes[codes['VARIABLE'] == 'MOTIVO_PRIORITARIO'][['CODE', 'VALUE_en']]
trips = trips.join(code_motive.set_index('CODE'),  on = 'MOTIVO_PRIORITARIO').reset_index()
trips.rename({"VALUE_en": "motive"}, axis = "columns", inplace = True)

# simplify motive
trips.loc[:,"motive_simple"] = trips.motive
trips.loc[(trips.MOTIVO_PRIORITARIO == 2) | (trips.MOTIVO_PRIORITARIO == 3), "motive_simple"] = "work"
trips.loc[(trips.MOTIVO_PRIORITARIO >= 5) & (trips.MOTIVO_PRIORITARIO <= 7), "motive_simple"] = "care"
trips.loc[(trips.MOTIVO_PRIORITARIO >= 8) & (trips.MOTIVO_PRIORITARIO <= 10), "motive_simple"] = "leisure / sports"
trips.loc[(trips.MOTIVO_PRIORITARIO == 1) | (trips.MOTIVO_PRIORITARIO == 11) | (trips.MOTIVO_PRIORITARIO == 12), "motive_simple"] = "other"

In [12]:
# daytime: rush hour, off-peak, low traffic
conditions = [
    ((trips.start_time >= time(7)) & (trips.start_time < time(9))) | ((trips.start_time >= time(17)) & (trips.start_time < time(20))),
     (trips.start_time >= time(9)) & (trips.start_time < time(17)),
     (trips.start_time >= time(20)) | (trips.start_time < time(7))
]
choices = ['rush hour (HVZ)', 'off-peak hour (NVZ)', 'low traffic time (SVZ)']
trips['daytime'] = np.select(conditions, choices, default= None)

In [13]:
# main activity
code_activity = codes[codes['VARIABLE'] == 'C8ACTIV'][['CODE', 'VALUE_en']]

trips = trips.join(code_activity.set_index('CODE'), on = 'C8ACTIV')

trips.rename({"VALUE_en": "activity"}, axis = "columns", inplace = True)

# simplify activity split
trips["activity_simple"] = trips.activity
trips.loc[trips.C8ACTIV <= 2, "activity_simple"] = "working"
trips.loc[(trips.C8ACTIV >= 3) & (trips.C8ACTIV <= 5), "activity_simple"] = "not working"
trips.loc[trips.C8ACTIV >= 7, "activity_simple"] = "domestic and care work"

In [14]:
# remove implausible data
trips.loc[trips.speed > 150, "speed"] = None

In [15]:
# create variable for rounded age
trips["rounded_age"] = 5 * (trips.EDAD_FIN / 5).round()

In [16]:
# age group
conditions = [
    trips.EDAD_FIN <= 18,
    (trips.EDAD_FIN > 18) & (trips.EDAD_FIN <= 25),
    (trips.EDAD_FIN > 25) & (trips.EDAD_FIN <= 55),
    trips.EDAD_FIN > 55]
choices = ['kids', 'young adults', 'adults', 'seniors']
trips['age_group'] = np.select(conditions, choices, default= None)

In [18]:
trips.drop(["index"], axis = 1, inplace = True)

In [19]:
trips.to_csv(r'..\data\processed\trips_custom_variables.csv', index = False)

In [20]:
trips.shape

(232600, 82)